In [26]:
# 1. Carga y exploración de datos


# Instalar PySpark en Google Colab
# Esta línea de comando instala la librería 'pyspark' en el entorno de Google Colab.
!pip install pyspark

# Importar las librerías necesarias
# 'SparkSession' es el punto de entrada para programar Spark con la API de DataFrame.
# 'StructType', 'StructField', etc., son tipos de datos de Spark. Aunque en este caso
# 'inferSchema=True' los detecta automáticamente, es una buena práctica tenerlos importados
# para cuando se defina un esquema de forma manual.
# 'pandas' se importa para posibles conversiones o visualizaciones, aunque no se usa en este bloque.
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType
import pandas as pd

# Iniciar una sesión de Spark
# Se crea una sesión de Spark, que es el punto de inicio para cualquier operación en Spark.
# `.builder` inicia el proceso de construcción de la sesión.
# `.appName("EvaluacionFinalSpark")` asigna un nombre a la aplicación Spark, útil para la monitorización en la UI de Spark.
# `.getOrCreate()` recupera una sesión existente o crea una nueva si no hay ninguna.
spark = SparkSession.builder \
    .appName("EvaluacionFinalSpark") \
    .getOrCreate()

# Cargar el dataset 'migraciones.csv'
# La función `spark.read.csv()` lee el archivo CSV.
# `header=True`: Le dice a Spark que la primera fila del archivo CSV contiene los nombres de las columnas, no los datos.
# `inferSchema=True`: Es una característica poderosa que hace que Spark escanee los datos para determinar automáticamente
# el tipo de datos de cada columna (por ejemplo, si es un número, una fecha o una cadena de texto). Esto simplifica el proceso de carga.
df_migraciones = spark.read.csv("migraciones.csv", header=True, inferSchema=True)

# Convertir el DataFrame a un RDD (Resilient Distributed Dataset)
# Los RDDs son la API de bajo nivel de Spark.
# Al llamar a `.rdd` sobre un DataFrame, se convierte a un RDD, lo que permite realizar
# operaciones de transformación y acción más detalladas y a bajo nivel si es necesario.
rdd_migraciones = df_migraciones.rdd

# ----------------- Exploración de datos -----------------

# 1. Mostrar las primeras filas del DataFrame
# `.show(5)` es una acción que muestra las 5 primeras filas del DataFrame.
# Es una forma rápida de inspeccionar el contenido y asegurarse de que los datos se han cargado correctamente.
print("Primeras 5 filas del DataFrame:")
df_migraciones.show(5)

# 2. Mostrar el esquema del DataFrame
# `.printSchema()` es una acción que imprime la estructura del DataFrame,
# mostrando el nombre de cada columna y su tipo de dato inferido.
# Esto es crucial para verificar que los tipos de datos se hayan reconocido correctamente.
print("Esquema del DataFrame:")
df_migraciones.printSchema()

# 3. Generar estadísticas descriptivas para las columnas numéricas
# `.describe()` es una acción que calcula y muestra un resumen estadístico de las columnas numéricas y de cadena,
# incluyendo el conteo, la media, la desviación estándar, el mínimo y el máximo.
# Es una excelente manera de obtener una visión general rápida de la distribución de los datos y detectar valores atípicos.
print("Estadísticas descriptivas de las columnas numéricas:")
df_migraciones.describe().show()

# Las estadísticas descriptivas ofrecen un resumen rápido de la distribución de los datos,
# como el conteo, la media, la desviación estándar, los valores mínimo y máximo.
# Es una excelente forma de detectar posibles anomalías o sesgos en los datos.

Primeras 5 filas del DataFrame:
+---+---------+---------------+----+---------+----------+-----------+---------------------+----------------------+----------------------+-----------------------+----------------+-----------------+
| ID|   Origen|        Destino| Año|    Razón|PIB_Origen|PIB_Destino|Tasa_Desempleo_Origen|Tasa_Desempleo_Destino|Nivel_Educativo_Origen|Nivel_Educativo_Destino|Población_Origen|Población_Destino|
+---+---------+---------------+----+---------+----------+-----------+---------------------+----------------------+----------------------+-----------------------+----------------+-----------------+
|  1|   México|           EEUU|2015|Económica|      8900|      62000|                  5.2|                   3.8|                   8.5|                   12.3|       125000000|        331000000|
|  2|    Siria|       Alemania|2016|Conflicto|      2500|      45000|                 15.4|                   4.5|                   6.2|                   14.1|        18000000|  

In [27]:
# 2. Procesamiento de datos con RDDs y DataFrames

# ----------------- Procesamiento con RDDs -----------------

# Convertir el DataFrame a RDD para las operaciones
# El RDD se crea a partir del DataFrame usando .rdd
# En este punto, 'rdd_migraciones' es un Resilient Distributed Dataset que contiene
# los mismos datos que el DataFrame, pero en un formato de bajo nivel.
# Cada elemento en el RDD es un objeto de tipo 'Row', que actúa como una fila de una tabla.
rdd_migraciones = df_migraciones.rdd

# Transformación: filter()
# 'filter()' es una transformación que crea un nuevo RDD, conservando solo los elementos que
# cumplen con una condición dada. En este caso, la condición es que el valor de la columna 'Año' sea 2017.
# La función lambda `lambda row: row['Año'] == 2017` se aplica a cada fila del RDD.
rdd_2017 = rdd_migraciones.filter(lambda row: row['Año'] == 2017)

# Acción: count()
# 'count()' es una acción que devuelve el número de elementos en el RDD.
# A diferencia de una transformación, una acción desencadena la ejecución de todas las
# transformaciones previas en el clúster de Spark y devuelve un resultado al driver.
print(f"Número de migraciones en el año 2017: {rdd_2017.count()}")

# Transformación: map()
# 'map()' es otra transformación que crea un nuevo RDD, aplicando una función a cada elemento del RDD original.
# Aquí, la función lambda `lambda row: (row['Origen'], row['Destino'])` transforma cada fila
# en una tupla que contiene solo los valores de las columnas 'Origen' y 'Destino'.
rdd_origen_destino = rdd_migraciones.map(lambda row: (row['Origen'], row['Destino']))

# Acción: take()
# 'take(n)' es una acción que devuelve una lista con los primeros 'n' elementos del RDD.
# Al igual que 'count()', esta acción fuerza la ejecución del DAG (Grafo Acíclico Dirigido) de Spark.
print("Primeras 5 tuplas de Origen-Destino:")
print(rdd_origen_destino.take(5))

# Transformación: flatMap()
# 'flatMap()' es una transformación similar a 'map()', pero puede generar 0 o más elementos
# de salida por cada elemento de entrada. Se usa comúnmente para aplanar colecciones anidadas.
# En este caso, se crea un RDD de las razones de migración, y la lista de un solo elemento
# se "aplana" en un RDD de elementos individuales.
rdd_razones = rdd_migraciones.flatMap(lambda row: [row['Razón']])

# Acción: collect() y countByValue()
# 'collect()' es una acción que devuelve todos los elementos del RDD como una lista en el driver.
# Se debe usar con precaución en datasets grandes, ya que puede agotar la memoria del driver.
# `Counter` de la librería `collections` se utiliza para contar la frecuencia de cada razón.
from collections import Counter
razones_conteo = Counter(rdd_razones.collect())
print("Conteo de razones de migración:")
print(razones_conteo)

# ----------------- Procesamiento con DataFrames -----------------
# El uso de DataFrames es la forma preferida para la mayoría de las tareas en Spark
# debido a su optimización y facilidad de uso.

# Filtrado: Migraciones con PIB de origen menor a 5000
# El filtrado en DataFrames es más legible que con RDDs. La sintaxis `df[columna] < valor`
# es similar a la de pandas y es muy intuitiva.
df_pobreza_origen = df_migraciones.filter(df_migraciones['PIB_Origen'] < 5000)
print("Migraciones con PIB de origen menor a 5000:")
df_pobreza_origen.show(5)

# Agregación: Conteo de migraciones por razón
# La agregación es una operación fundamental. Primero, se usa `groupBy()` para agrupar las filas
# por la columna 'Razón'. Luego, `.agg(count("*").alias("Total_Migraciones"))` aplica la función
# de agregación `count()` a cada grupo y le asigna el alias 'Total_Migraciones' al resultado.
from pyspark.sql.functions import count
df_razones_agregado = df_migraciones.groupBy("Razón").agg(count("*").alias("Total_Migraciones"))
print("Conteo total de migraciones por razón:")
df_razones_agregado.show()

# Ordenamiento: Ordenar las razones por el total de migraciones de forma descendente
# La función `.orderBy()` se utiliza para ordenar el DataFrame.
# La función `desc()` del módulo `pyspark.sql.functions` especifica un orden descendente.
# Las operaciones de ordenamiento son costosas ya que requieren un 'shuffle' de los datos entre los nodos del clúster.
from pyspark.sql.functions import desc
df_razones_ordenado = df_razones_agregado.orderBy(desc("Total_Migraciones"))
print("Principales razones de migración:")
df_razones_ordenado.show()

# ----------------- Escribir en formato Parquet -----------------

# Se guarda el DataFrame de migraciones en formato Parquet.
# Parquet es un formato de archivo columnar optimizado para el ecosistema de Big Data.
# Es más eficiente para la lectura y el almacenamiento que CSV.
# `.write` es el objeto de escritura del DataFrame.
# `.mode("overwrite")` indica que si el archivo de salida ya existe, debe ser sobrescrito.
# `.parquet("migraciones_parquet")` guarda el DataFrame en el formato Parquet en la ruta especificada.
print("Guardando el DataFrame en formato Parquet...")
df_migraciones.write.mode("overwrite").parquet("migraciones_parquet")
print("Archivo Parquet guardado exitosamente.")

Número de migraciones en el año 2017: 1
Primeras 5 tuplas de Origen-Destino:
[('México', 'EEUU'), ('Siria', 'Alemania'), ('Venezuela', 'Colombia'), ('India', 'Emiratos Árabes'), ('Argentina', 'España')]
Conteo de razones de migración:
Counter({'Económica': 2, 'Conflicto': 1, 'Política': 1, 'Laboral': 1})
Migraciones con PIB de origen menor a 5000:
+---+------+---------------+----+---------+----------+-----------+---------------------+----------------------+----------------------+-----------------------+----------------+-----------------+
| ID|Origen|        Destino| Año|    Razón|PIB_Origen|PIB_Destino|Tasa_Desempleo_Origen|Tasa_Desempleo_Destino|Nivel_Educativo_Origen|Nivel_Educativo_Destino|Población_Origen|Población_Destino|
+---+------+---------------+----+---------+----------+-----------+---------------------+----------------------+----------------------+-----------------------+----------------+-----------------+
|  2| Siria|       Alemania|2016|Conflicto|      2500|      45000|  

In [28]:
# 3. Consultas con Spark SQL

# ----------------- Consultas con Spark SQL -----------------

# Registrar el DataFrame como una tabla temporal
# La función `createOrReplaceTempView()` crea una vista temporal del DataFrame en la sesión de Spark.
# Esto es un paso crucial porque nos permite ejecutar consultas SQL estándar sobre los datos del DataFrame.
# La vista temporal solo existe mientras la sesión de Spark está activa y es accesible usando el nombre "migraciones".
df_migraciones.createOrReplaceTempView("migraciones")
print("DataFrame 'migraciones' registrado como tabla temporal.")

# Consulta 1: Principales países de origen
# Se utiliza Spark SQL para encontrar los países de origen más comunes.
# La consulta sigue un flujo lógico:
# - SELECT Origen, COUNT(*): Se seleccionan los países de origen y se cuenta cuántas veces aparecen.
# - FROM migraciones: Se especifica la tabla temporal a consultar.
# - GROUP BY Origen: Se agrupan las filas que tienen el mismo país de origen para poder contarlas.
# - ORDER BY Total_Migraciones DESC: Se ordena el resultado de forma descendente para tener el mayor conteo primero.
# - LIMIT 5: Se limita el resultado a los 5 países principales.
sql_principales_origenes = spark.sql("""
    SELECT Origen, COUNT(*) AS Total_Migraciones
    FROM migraciones
    GROUP BY Origen
    ORDER BY Total_Migraciones DESC
    LIMIT 5
""")
print("Principales países de origen:")
sql_principales_origenes.show()

# Consulta 2: Principales países de destino
# Esta consulta es similar a la anterior, pero se enfoca en los países de destino más frecuentes.
# La lógica es idéntica: agrupar por el país de destino, contar las ocurrencias, ordenar y tomar los 5 primeros.
sql_principales_destinos = spark.sql("""
    SELECT Destino, COUNT(*) AS Total_Migraciones
    FROM migraciones
    GROUP BY Destino
    ORDER BY Total_Migraciones DESC
    LIMIT 5
""")
print("Principales países de destino:")
sql_principales_destinos.show()

# ----------------- Consulta Corregida con Caracteres Especiales -----------------

# Análisis 3: Principales razones de migración
# **NOTA DE CORRECCIÓN:** El nombre de la columna 'Razón' contiene un carácter especial ('ó').
# Para que Spark SQL interprete correctamente el nombre de la columna y no genere un error de sintaxis,
# es necesario rodear el nombre con comillas invertidas (backticks: ` `).
# Esto le dice a Spark que trate el texto como un identificador de columna literal.
sql_razones_por_region = spark.sql("""
    SELECT `Razón`, COUNT(*) AS Total_Migraciones
    FROM migraciones
    GROUP BY `Razón`
    ORDER BY Total_Migraciones DESC
""")

print("Principales razones de migración (consulta corregida):")
sql_razones_por_region.show()

DataFrame 'migraciones' registrado como tabla temporal.
Principales países de origen:
+---------+-----------------+
|   Origen|Total_Migraciones|
+---------+-----------------+
|Argentina|                1|
|    Siria|                1|
|    India|                1|
|Venezuela|                1|
|   México|                1|
+---------+-----------------+

Principales países de destino:
+---------------+-----------------+
|        Destino|Total_Migraciones|
+---------------+-----------------+
|         España|                1|
|Emiratos Árabes|                1|
|       Alemania|                1|
|           EEUU|                1|
|       Colombia|                1|
+---------------+-----------------+

Principales razones de migración (consulta corregida):
+---------+-----------------+
|    Razón|Total_Migraciones|
+---------+-----------------+
|Económica|                2|
| Política|                1|
|Conflicto|                1|
|  Laboral|                1|
+---------+-----------

In [29]:
# NOTA IMPORTANTE: Si obtienes un error de 'AssertionError: SparkContext is not None',
# significa que la sesión de Spark ha sido detenida. Por favor, asegúrate de
# ejecutar la celda de "Iniciar una sesión de Spark" antes de ejecutar esta celda.

# ----------------- Aplicación de MLlib -----------------

# Importar las librerías necesarias de MLlib
# VectorAssembler: Combina múltiples columnas de características en un solo vector. Esto es un requisito de MLlib.
# StringIndexer: Transforma columnas de tipo string a valores numéricos.
# LogisticRegression: El algoritmo de clasificación que se utilizará.
# BinaryClassificationEvaluator: Se usa para evaluar el rendimiento de un modelo de clasificación binaria.
# when: Una función de Spark SQL para crear una nueva columna con una lógica condicional.
from pyspark.ml.feature import VectorAssembler, StringIndexer
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.sql.functions import when

# Paso 1: Convertir la variable 'Razón' en una etiqueta numérica (label)
# El problema de Machine Learning es predecir si un evento es una migración (1) o no (0).
# Creamos una nueva columna `es_migracion` para que sea la variable objetivo (`label`).
# Usamos `when()` para asignar 1.0 a todas las filas, ya que todo el dataset son migraciones.
# En un escenario real, necesitarías datos de "no-migraciones" para tener la clase 0.
df_ml = df_migraciones.withColumn("es_migracion", when(df_migraciones["Razón"].isNotNull(), 1.0).otherwise(0.0))

# Paso 2: Seleccionar las características numéricas para el modelo
# Las características (`features`) son las variables predictoras que el modelo usará.
# Elegimos las columnas socioeconómicas que son relevantes para la predicción de migración.
feature_cols = [
    'PIB_Origen', 'PIB_Destino',
    'Tasa_Desempleo_Origen', 'Tasa_Desempleo_Destino',
    'Población_Origen', 'Población_Destino'
]

# Paso 3: Usar VectorAssembler para combinar las características en un solo vector
# `VectorAssembler` es una transformación que convierte las columnas de características
# en una sola columna de tipo "vector", que es el formato que los modelos de MLlib requieren.
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
# `transform()` aplica la transformación al DataFrame.
df_vectorized = assembler.transform(df_ml)

# Paso 4: Dividir los datos en conjuntos de entrenamiento y prueba
# Se usa `randomSplit()` para dividir el DataFrame en dos subconjuntos:
# - `training_data`: Para entrenar el modelo (70% de los datos).
# - `test_data`: Para evaluar el rendimiento del modelo con datos que no ha visto (30%).
# `seed=42` garantiza que la división sea reproducible.
(training_data, test_data) = df_vectorized.randomSplit([0.7, 0.3], seed=42)
print(f"Tamaño del conjunto de entrenamiento: {training_data.count()}")
print(f"Tamaño del conjunto de prueba: {test_data.count()}")

# Paso 5: Entrenar el modelo de Regresión Logística
# `LogisticRegression()` crea el modelo, especificando qué columnas son las características (`featuresCol`)
# y cuál es la variable objetivo (`labelCol`).
# `.fit()` entrena el modelo usando el conjunto de datos de entrenamiento.
lr = LogisticRegression(labelCol="es_migracion", featuresCol="features")
print("Entrenando el modelo de Regresión Logística...")
model = lr.fit(training_data)
print("Modelo entrenado.")

# Paso 6: Realizar predicciones en el conjunto de prueba
# `transform()` usa el modelo entrenado para hacer predicciones en el conjunto de prueba.
# El resultado es un nuevo DataFrame que incluye las columnas originales más las predicciones
# y las probabilidades calculadas por el modelo.
predictions = model.transform(test_data)
print("Primeras 5 predicciones del modelo:")
# Se muestran las columnas originales de la etiqueta, la predicción y la probabilidad.
# `probability` muestra las probabilidades para ambas clases ([prob_clase_0, prob_clase_1]).
predictions.select("es_migracion", "prediction", "probability").show(5)

# Paso 7: Evaluar el modelo
# `BinaryClassificationEvaluator` se usa para medir la precisión del modelo de clasificación binaria.
# La métrica `areaUnderROC` es una medida común que evalúa la capacidad del modelo para distinguir
# entre las dos clases. Un valor más cercano a 1.0 es mejor.
evaluator = BinaryClassificationEvaluator(labelCol="es_migracion")
area_under_roc = evaluator.evaluate(predictions)
print(f"Área bajo la curva ROC (AUC): {area_under_roc}")

# Análisis de la evaluación
'''
Análisis de precisión del modelo:
- La métrica 'Área bajo la curva ROC (AUC)' mide la capacidad del modelo para distinguir entre las clases.
- Un valor de AUC de 1.0 indica un modelo perfecto, mientras que un valor de 0.5 indica un rendimiento aleatorio.
- El resultado del modelo muestra un AUC de 1.0. Esto parece un rendimiento perfecto, pero es un claro síntoma de sobreajuste.
- Este sobreajuste ocurre porque el dataset es extremadamente pequeño (solo 5 registros) y solo contiene una clase ('es_migracion' = 1.0). El modelo simplemente "memoriza" el patrón y predice la única clase que ha visto.
- En un escenario real, un resultado así es sospechoso y requeriría más datos y un conjunto de prueba más grande para ser validado.
'''

# Detener la sesión de Spark
# Es una buena práctica detener la sesión de Spark al final del script para liberar los recursos del clúster.
spark.stop()
print("Sesión de Spark detenida.")

Tamaño del conjunto de entrenamiento: 4
Tamaño del conjunto de prueba: 1
Entrenando el modelo de Regresión Logística...
Modelo entrenado.
Primeras 5 predicciones del modelo:
+------------+----------+-----------+
|es_migracion|prediction|probability|
+------------+----------+-----------+
|         1.0|       1.0|  [0.0,1.0]|
+------------+----------+-----------+

Área bajo la curva ROC (AUC): 1.0
Sesión de Spark detenida.
